In [ ]:
import pyspark as ps
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from scipy import stats
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import time

In [ ]:
## config

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 300)

In [ ]:
# iniciando variaveis;

df = pd.DataFrame()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json',
    'Accept-Language': 'pt-BR,en;q=0.5',
    'Connection': 'keep-alive',
    'Referer': 'https://www.webmotors.com.br/',
    'Origin': 'https://www.webmotors.com.br'
}

### Objetivo: ler os dados de uma pesquisa na webmotors para análise de dados;

In [ ]:
# API headers/call test;

sesh = requests.Session()
r2 = sesh.get('https://www.webmotors.com.br', headers = headers)
r2.status_code

<h2> API Call Function </h2>

In [ ]:
raw_result = {}

def api_call(page_num):

    assert page_num > 0

    url = f"""https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Festoque%3Flkid%3D1022%26tipoveiculo%3Dcarros-usados&actualPage={page_num}&displayPerPage=1000&order=1&showMenu=true&showCount=true&showBreadCrumb=true&testAB=false&returnUrl=false&pandora=false&opcionais=Bancos%20em%20couro"""


    session = requests.Session()
    r = session.get(url, headers=headers)
    

    df_page = pd.json_normalize(r.json()['SearchResults'])
    raw_result[page_num] = r.json()

    return df_page


In [ ]:
# com marca/modelo:


raw_result = {}

def api_call_byModel(page_num, marca = '', modelo = ''):

    assert page_num > 0


    modelo = modelo.upper()
    marca = marca.upper()

    url = f"""https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros-usados%2Festoque%3Flkid%3D1022%26tipoveiculo%3Dcarros-usados&actualPage={page_num}&marca1={marca}&modelo1={modelo}&displayPerPage=1000&order=1&showMenu=true&showCount=true&showBreadCrumb=true&testAB=false&returnUrl=false&pandora=false&opcionais=Bancos%20em%20couro"""


    session = requests.Session()
    r = session.get(url, headers=headers)
    

    df_page = pd.json_normalize(r.json()['SearchResults'])
    raw_result[page_num] = r.json()

    return df_page




In [ ]:
df_virtus = api_call_byModel(1, 'VOLKSWAGEN', 'VIRTUS')



# Scraping all search pages;


In [ ]:

for page_num in range(1, 5):
    

    df_page_n = api_call(page_num)
    assert df_page_n.empty == False

    df = pd.concat([df, df_page_n])


In [ ]:
df.head(3)

In [ ]:
raw_result

<h2> EDA </h2> 

In [ ]:
print(f"Número de linhas no df: {df.shape[0]}")

In [ ]:
df.iloc[0]

<h3> COLUMNS OF INTEREST </h3>

In [ ]:
import re
[re.search('Specification.+', x).group(0) for x in df.columns if re.search('Specification', x)]

In [ ]:
cols = [
        'UniqueId', 
        'ProductCode', 
        'LongComment',
        'FipePercent',
        'GoodDeal',
        'HotDeal',
        'Specification.Title',
        'Specification.Make.id',
        'Specification.Make.Value',
        'Specification.Model.id',
        'Specification.Model.Value',
        'Specification.Version.id',
        'Specification.Version.Value',
        'Specification.YearFabrication',
        'Specification.YearModel',
        'Specification.Odometer',
        'Specification.Transmission',
        'Specification.NumberPorts',
        'Specification.BodyType',
        'Specification.VehicleAttributes',
        'Specification.Armored',
        'Specification.Color.IdPrimary',
        'Specification.Color.Primary',
        'Seller.SellerType',                                               
        'Seller.City',
        'Seller.State',
        'Prices.Price',
        'Prices.SearchPrice'                        
]

df = df[cols]

# rename of cols
df.columns = [string.replace('.', '_').replace('Specification_', '') for string in cols]

In [ ]:
df.head(3)

In [ ]:
df.dtypes

In [ ]:
## NA Values
df.isna().sum()[df.isna().sum()>0]

In [ ]:
df = df.fillna({'Version_Value' : '0.0'})

In [ ]:
df.Version_Value

In [ ]:
## Feature Engineering

# Motor" 1.0, 1.6, 2.0 etc. 
p = re.compile('[0-9]\.[0-9]')
motor_cil = df.Version_Value.apply(lambda x: p.search(x).group(0) if p.search(x) else 0)
df['motor'] = motor_cil


# Combustível; 
p = re.compile('flex|diesel|gasolina|alcool|álcool')
df.Version_Value



<h3> Case Studies: cars statics under xx.xxx price </h3>

In [ ]:
df_100k = df[df['Prices_Price'] <= 85000]

df_100k.describe().applymap(lambda x: round(x, 2))

In [ ]:
df_100k.head(3)

#### Quais fabricantes possuem, em média, carros mais novos, até a faixa dos 100k? 


In [ ]:

df_100k.groupby(['Make_Value'])[['Model_Value', 'YearModel', 'Odometer', 'Prices_Price']] \
    .agg(
        {'Model_Value': 'count',
        'YearModel': ['mean', 'min', 'max'],
        'Odometer': ['mean', 'min', 'max'],
        'Prices_Price': ['mean', 'min', 'max']
        }
        ).sort_values([('YearModel', 'mean'), ('Odometer', 'mean')], ascending=[False, False])

#### Lista dos modelos mais encontrados, por fabricante

In [ ]:
df_100k.groupby(['Make_Value', 'Model_Value'])[['Model_Value', 'YearModel', 'Odometer', 'Prices_Price']] \
    .agg(
        {'Model_Value': 'count',
        'YearModel': ['mean', 'min', 'max'],
        'Odometer': ['mean', 'min', 'max'],
        'Prices_Price': ['mean', 'min', 'max']
        }
        ).sort_values(['Make_Value', ('Model_Value', 'count')], ascending=[True, False])

In [ ]:
df_100k.groupby(['Make_Value', 'Model_Value'])[['Model_Value', 'YearModel', 'Odometer', 'Prices_Price']] \
    .agg({
        'Model_Value': [('model_count', 'count')],
        'YearModel': ['mean', 'min', 'max'],
        'Odometer': ['mean', 'min', 'max'],
        'Prices_Price': ['mean', 'min', 'max']}).sort_values(['Make_Value', ('Model_Value', 'model_count')], ascending=[True, False]).applymap(lambda x: round(x, 2) if isinstance(x, float)  else x)

In [ ]:
df.Seller_State[0].values

### My Search 

In [ ]:
#### carros até 80k e no estado de SP
df_70k = df[(df.Prices_Price <= 80000)
            & (df.YearModel >= 2018)
            & (df.Seller_State == 'São Paulo (SP)')
            & (df.Odometer <=70000)
            ]



df_70k.groupby(['Make_Value', 'Model_Value'])[['Model_Value', 'YearModel', 'Odometer', 'Prices_Price']] \
    .agg(
        {'Model_Value': 'count',
        'YearModel': ['mean', 'max', 'min'],
        'Odometer': ['mean', 'min', 'max'],
        'Prices_Price': ['mean', 'min', 'max']
        }
        ).sort_values(['Make_Value', ('YearModel', 'mean'), ('Prices_Price', 'mean')], ascending=[True, False, False])

In [ ]:
df.head(3)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

df_70k_model_count = df_70k[['Model_Value', 'YearModel']].groupby('Model_Value').agg( count = ('YearModel', 'count')).sort_values('count')


plt.figure(figsize=(12,10))
sns.barplot(df_70k_model_count, x = 'count', y = df_70k_model_count.index, width = 0.7)

plt.xticks(rotation=0)
plt.title("Carros até R$ 80k <70.000 km & >=2018")
plt.ylabel('Car Model')
plt.xlabel('Units Listed')

#plt.subplots_adjust(bottom=0.2) 
plt.show()

In [ ]:
df_70k